# 01 - Exploratory Data Analysis of RAW input data

In [1]:
# Import necessary libraries
import rasterio
from rasterio.enums import Resampling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from skimage.feature import graycomatrix, graycoprops
from rasterio.warp import reproject, calculate_default_transform
from typing import Tuple, Dict, Any

# Set plotting style
sns.set(style="whitegrid")

In [ ]:
# Utils functions

def plot_image(image: np.ndarray, title: str, cmap: str = 'viridis') -> None:
    """Plot a single band image.

    Args:
        image (np.ndarray): The image array to plot.
        title (str): Title of the plot.
        cmap (str): Colormap to use for the plot.
    """
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap=cmap)
    plt.colorbar()
    plt.title(title)
    plt.axis('off')
    plt.show()

def plot_histogram(image: np.ndarray, title: str) -> None:
    """Plot a histogram of pixel values in an image.

    Args:
        image (np.ndarray): The image array to plot.
        title (str): Title of the histogram.
    """
    plt.figure(figsize=(10, 6))
    plt.hist(image.flatten(), bins=50, color='c', edgecolor='k', alpha=0.7)
    plt.title(title)
    plt.xlabel('Pixel Values')
    plt.ylabel('Frequency')
    plt.show()

def summarize_image(image: np.ndarray, name: str) -> None:
    """Print summary statistics of an image.

    Args:
        image (np.ndarray): The image array to summarize.
        name (str): Name of the image.
    """
    print(f"Summary statistics for {name}:")
    print(f" - Shape: {image.shape}")
    print(f" - Min value: {np.min(image)}")
    print(f" - Max value: {np.max(image)}")
    print(f" - Mean value: {np.mean(image)}")
    print(f" - Standard deviation: {np.std(image)}\n")

def resample_image(src: rasterio.io.DatasetReader, target_transform: rasterio.Affine, 
                   target_shape: Tuple[int, int]) -> np.ndarray:
    """Resample an image to a target resolution and shape.

    Args:
        src (rasterio.io.DatasetReader): Source dataset reader.
        target_transform (rasterio.Affine): Target affine transform.
        target_shape (Tuple[int, int]): Target shape.

    Returns:
        np.ndarray: Resampled image.
    """
    data = src.read(
        out_shape=(
            src.count,
            target_shape[0],
            target_shape[1]
        ),
        resampling=Resampling.bilinear
    )
    return data.squeeze()


def reproject_resample(src_path: str, target_crs: str, 
                       target_transform: rasterio.Affine, target_shape: Tuple[int, int]) -> Tuple[np.ndarray, Dict[str, Any]]:
    """Reproject and resample an image to match a target CRS and resolution.

    Args:
        src_path (str): Path to the source image.
        target_crs (str): Target coordinate reference system.
        target_transform (rasterio.Affine): Target affine transform.
        target_shape (Tuple[int, int]): Target shape.

    Returns:
        Tuple[np.ndarray, Dict[str, Any]]: Resampled image and metadata.
    """
    with rasterio.open(src_path) as src:
        if src.crs != target_crs:
            transform, width, height = calculate_default_transform(
                src.crs, target_crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': target_crs,
                'transform': transform,
                'width': width,
                'height': height
            })

            data = np.empty((src.count, target_shape[0], target_shape[1]), dtype=src.dtypes[0])
            reproject(
                source=rasterio.band(src, 1),
                destination=data,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=target_transform,
                dst_crs=target_crs,
                resampling=Resampling.bilinear
            )
        else:
            kwargs = src.meta.copy()
            data = src.read(
                out_shape=(
                    src.count,
                    target_shape[0],
                    target_shape[1]
                ),
                resampling=Resampling.bilinear
            )
    return data.squeeze(), kwargs

def plot_pca(vv_band: np.ndarray, hand_resampled: np.ndarray, mask_resampled: np.ndarray, sample_size: int = 100000) -> None:
    """Perform PCA on VV Band and HAND data and plot the results.

    Args:
        vv_band (np.ndarray): VV Band image array.
        hand_resampled (np.ndarray): Resampled HAND image array.
        mask_resampled (np.ndarray): Resampled binary mask array.
        sample_size (int): Number of pixels to sample for PCA.
    """
    np.random.seed(42)
    
    vv_flat = vv_band.flatten()
    hand_flat = hand_resampled.flatten()
    mask_flat = mask_resampled.flatten()
    
    combined = np.vstack((vv_flat, hand_flat)).T
    indices = np.random.choice(combined.shape[0], sample_size, replace=False)
    combined_sampled = combined[indices, :]
    mask_sampled = mask_flat[indices]
    
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(combined_sampled)

    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], c=mask_sampled, cmap='coolwarm', alpha=0.5)
    plt.title('PCA of VV Band and HAND')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.colorbar(scatter, label='Binary Mask')
    plt.show()

def plot_texture_features(image: np.ndarray, distances: list[int], angles: list[float]) -> None:
    """Plot texture features of an image using GLCM.

    Args:
        image (np.ndarray): Image array to analyze.
        distances (list[int]): List of distances for GLCM.
        angles (list[float]): List of angles for GLCM.
    """
    image_uint8 = (image / image.max() * 255).astype('uint8')
    
    glcm = graycomatrix(image_uint8, distances=distances, angles=angles, symmetric=True, normed=True)
    features = {
        'Contrast': graycoprops(glcm, 'contrast').flatten(),
        'Dissimilarity': graycoprops(glcm, 'dissimilarity').flatten(),
        'Homogeneity': graycoprops(glcm, 'homogeneity').flatten(),
        'Energy': graycoprops(glcm, 'energy').flatten(),
        'Correlation': graycoprops(glcm, 'correlation').flatten(),
        'ASM': graycoprops(glcm, 'ASM').flatten()
    }

    plt.figure(figsize=(15, 10))
    for i, (feature, values) in enumerate(features.items()):
        plt.subplot(2, 3, i + 1)
        sns.barplot(x=['d=1, a=0', 'd=1, a=pi/4', 'd=1, a=pi/2', 'd=1, a=3pi/4'], y=values)
        plt.title(feature)
        plt.xlabel('Parameter Combination')
        plt.ylabel('Value')

    plt.tight_layout()
    plt.show()
